<a href="https://colab.research.google.com/github/ShikharV010/gist_daily_runs/blob/main/SalesDialierAPICall_yesterday.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
!pip install sqlalchemy psycopg2-binary

In [5]:
"""
JustCall Sales-Dialer ingestion
• Robust 429 handling (epoch vs. delta headers)
• Precise per-minute pacing (you choose the burst size)
• Continuous progress logging
"""

import requests, json, time, pandas as pd
from datetime import datetime, timedelta

# ────────────────────────────────────────────────────────────
# 1) CONFIG – adjust to your account
# ────────────────────────────────────────────────────────────
API_KEY    = "cc7718b616f3be5e663be9f132548cbf083fc5e9"
API_SECRET = "1f26c3c1e9bbf56324f5f9ddb70bab81b42cff38"

MAX_CALLS_PER_MIN   = 28     # leave head-room under plan burst (30/60/90)
MAX_RETRIES         = 8
BACKOFF_FACTOR      = 2      # 1,2,4,8…

DATE_FROM = (datetime.now() - timedelta(days=1)).strftime("%Y-%m-%d")
DATE_TO   = (datetime.now() - timedelta(days=1)).strftime("%Y-%m-%d")

# ────────────────────────────────────────────────────────────
# 2) SESSION
# ────────────────────────────────────────────────────────────
session = requests.Session()
session.auth = (API_KEY, API_SECRET)
session.headers.update({"Authorization": f"{API_KEY}:{API_SECRET}"})
REQUEST_TIMEOUT = 15     # seconds

# ────────────────────────────────────────────────────────────
# 3) Rate-limit helpers
# ────────────────────────────────────────────────────────────
def secs_from_header(raw: str | None) -> int | None:
    """Return number of seconds to wait, or None if header absent/invalid."""
    if not (raw and raw.isdigit()):
        return None
    val = int(raw)
    return val - int(time.time()) if val > 86_400 else val   # epoch vs delta

def respect_burst(window_start, calls_made):
    """Block if we already made MAX_CALLS_PER_MIN requests in this minute."""
    if calls_made and calls_made % MAX_CALLS_PER_MIN == 0:
        elapsed = time.time() - window_start
        wait = max(0, 60 - elapsed)
        if wait:
            print(f"↪︎ pacing – sleeping {wait:.1f}s to stay under burst limit")
            time.sleep(wait)
        return time.time()   # reset window start
    return window_start

# ────────────────────────────────────────────────────────────
# 4) GET with smart 429 handling
# ────────────────────────────────────────────────────────────
def safe_get(url):
    for attempt in range(MAX_RETRIES):
        r = session.get(url, timeout=REQUEST_TIMEOUT)
        if r.status_code != 429:
            r.raise_for_status()
            return r.json()

        wait = secs_from_header(r.headers.get("X-Rate-Limit-Burst-Reset") or
                                r.headers.get("Retry-After"))
        if wait is None:
            wait = BACKOFF_FACTOR ** attempt
        wait = max(wait, 1)
        print(f"429 → wait {wait}s (retry {attempt+1}/{MAX_RETRIES})")
        time.sleep(wait)
    raise RuntimeError(f"gave up after {MAX_RETRIES} retries → {url}")

# ────────────────────────────────────────────────────────────
# 5) API wrappers
# ────────────────────────────────────────────────────────────
def list_calls(date_from=DATE_FROM, date_to=DATE_TO):
    url = "https://api.justcall.io/v1/autodialer/calls/list"
    all_calls, page = [], 1
    while True:
        payload = {"start_date": date_from, "end_date": date_to, "page": page}
        r = session.post(url, json=payload, timeout=REQUEST_TIMEOUT)
        r.raise_for_status()
        data = r.json().get("data", [])
        if not data:
            break
        all_calls.extend(data)
        print(f"• got page {page} ({len(data)} calls)")
        page += 1
    return all_calls

def call_detail(call_id):
    url = f"https://api.justcall.io/v2.1/sales_dialer/calls/{call_id}"
    return safe_get(url).get("data", {})

# ────────────────────────────────────────────────────────────
# 6) Flatten to DataFrame
# ────────────────────────────────────────────────────────────
def flatten(detail_list):
    stamp = datetime.now().strftime("%Y-%m-%d")
    rows = [{
        "call_id":          d.get("call_id"),
        "campaign":         json.dumps(d.get("campaign", {})),
        "contact_id":       d.get("contact_id"),
        "contact_number":   d.get("contact_number"),
        "contact_name":     d.get("contact_name", ""),
        "contact_email":    d.get("contact_email", ""),
        "agent_name":       d.get("agent_name", ""),
        "agent_email":      d.get("agent_email", ""),
        "call_date":        d.get("call_date"),
        "call_time":        d.get("call_time"),
        "call_info":        json.dumps(d.get("call_info", {})),
        "date_ingested":    stamp
    } for d in detail_list]
    return pd.DataFrame(rows)

# ────────────────────────────────────────────────────────────
# 7) Main workflow
# ────────────────────────────────────────────────────────────
def run_ingestion():
    print(f"\n⏳ Fetching IDs {DATE_FROM} → {DATE_TO} …")
    ids = list_calls()
    total = len(ids)
    print(f"✓ {total} IDs found\n")

    details, missing = [], []
    window_start = time.time()

    for idx, c in enumerate(ids, 1):
        window_start = respect_burst(window_start, idx)
        cid = c["call_id"]
        try:
            d = call_detail(cid)
            if d:
                details.append(d)
            else:
                missing.append(cid)
            if idx % 25 == 0 or idx == total:
                print(f"  progress {idx}/{total}")
        except Exception as e:
            print(f"⚠️  {cid} skipped → {e}")
            missing.append(cid)

    df = flatten(details)
    print(f"\n🏁 finished – {len(df)}/{total} rows")
    if missing:
        print(f"  still missing {len(missing)} IDs → {missing[:10]} …")
    return df

# ────────────────────────────────────────────────────────────
if __name__ == "__main__":
    df_calls = run_ingestion()
    # df_calls.to_csv("justcall_calls.csv", index=False)



⏳ Fetching IDs 2025-07-18 → 2025-07-18 …
• got page 1 (50 calls)
• got page 2 (50 calls)
• got page 3 (50 calls)
• got page 4 (50 calls)
• got page 5 (50 calls)
• got page 6 (50 calls)
• got page 7 (50 calls)
• got page 8 (50 calls)
• got page 9 (50 calls)
• got page 10 (50 calls)
• got page 11 (50 calls)
• got page 12 (50 calls)
• got page 13 (50 calls)
• got page 14 (50 calls)
• got page 15 (50 calls)
• got page 16 (30 calls)
✓ 780 IDs found

  progress 25/780
↪︎ pacing – sleeping 55.7s to stay under burst limit
  progress 50/780
↪︎ pacing – sleeping 56.5s to stay under burst limit
  progress 75/780
↪︎ pacing – sleeping 56.4s to stay under burst limit
  progress 100/780
↪︎ pacing – sleeping 56.4s to stay under burst limit
  progress 125/780
↪︎ pacing – sleeping 56.5s to stay under burst limit
  progress 150/780
↪︎ pacing – sleeping 56.7s to stay under burst limit
  progress 175/780
↪︎ pacing – sleeping 56.7s to stay under burst limit
  progress 200/780
↪︎ pacing – sleeping 56.4s to 

In [10]:
# df_calls_filtered = df_calls[df_calls['call_date'] == '2025-07-10']
# display(df_calls_filtered)


,call_id,campaign,contact_id,contact_number,contact_name,contact_email,agent_name,agent_email,call_date,call_time,call_info,date_ingested


In [6]:
import pandas as pd
import sqlalchemy                       # <- new (needed only if you add dtype=)
from sqlalchemy import create_engine, text
from datetime import datetime

# ───────────── DB config ─────────────
engine = create_engine(
    "postgresql://airbyte_user:airbyte_user_password@"
    "gw-postgres-dev.celzx4qnlkfp.us-east-1.rds.amazonaws.com:5432/gw_prod"
)
TABLE_SCHEMA = "gist"
TABLE_NAME   = "gist_salesdialercalldetails"
VIEW_NAME    = "vw_salesdialercalldetails"

# ───────────── DataFrame from ingestion ─────────────
df = df_calls.copy()                    # <-- the only change
if df.empty:
    print("🛑 No new data to insert."); raise SystemExit

df["date_ingested"] = datetime.utcnow().date()   # keep stamp in UTC

try:
    # 1️⃣  pull existing call_ids (small result set, OK for now)
    with engine.connect() as conn:
        existing = {row[0] for row in conn.execute(
            text(f"SELECT call_id FROM {TABLE_SCHEMA}.{TABLE_NAME}")
        )}
    print(f"📦 existing rows in DB: {len(existing)}")

    # 2️⃣  filter out duplicates
    df_new = df[~df["call_id"].isin(existing)]
    print(f"🆕 rows to insert: {len(df_new)}")

    # 3️⃣  append
    if not df_new.empty:
        df_new.to_sql(
            name=TABLE_NAME,
            con=engine,
            schema=TABLE_SCHEMA,
            if_exists="append",
            index=False,
            method="multi"
            # dtype={"campaign": sqlalchemy.dialects.postgresql.JSONB,
            #        "call_info": sqlalchemy.dialects.postgresql.JSONB}
        )
        print("✅ new rows appended.")
    else:
        print("🛑 nothing new to append.")

except Exception as e:
    # table missing → create from scratch
    print(f"📭 table absent or error querying it → creating afresh.\n{e}")
    df.to_sql(
        name=TABLE_NAME,
        con=engine,
        schema=TABLE_SCHEMA,
        if_exists="replace",
        index=False,
        method="multi"
    )
    print(f"✅ table {TABLE_SCHEMA}.{TABLE_NAME} created.")

# 4️⃣  make / refresh view
with engine.begin() as conn:
    conn.execute(text(f"""
        CREATE OR REPLACE VIEW {TABLE_SCHEMA}.{VIEW_NAME} AS
        SELECT *
        FROM   {TABLE_SCHEMA}.{TABLE_NAME};
    """))
print(f"🪟 view {TABLE_SCHEMA}.{VIEW_NAME} refreshed.")
engine.dispose()


📦 existing rows in DB: 7812
🆕 rows to insert: 508
✅ new rows appended.
🪟 view gist.vw_salesdialercalldetails refreshed.


TEST

In [2]:
# """
# JustCall Sales-Dialer ingestion
# • Robust 429 handling (epoch vs. delta headers)
# • Precise per-minute pacing (you choose the burst size)
# • Continuous progress logging
# """

# import requests, json, time, pandas as pd
# from datetime import datetime, timedelta

# # ────────────────────────────────────────────────────────────
# # 1) CONFIG – adjust to your account
# # ────────────────────────────────────────────────────────────
# API_KEY    = "cc7718b616f3be5e663be9f132548cbf083fc5e9"
# API_SECRET = "1f26c3c1e9bbf56324f5f9ddb70bab81b42cff38"

# MAX_CALLS_PER_MIN   = 28     # leave head-room under plan burst (30/60/90)
# MAX_RETRIES         = 8
# BACKOFF_FACTOR      = 2      # 1,2,4,8…

# DATE_FROM = "2025-07-18"#(datetime.now() - timedelta(days=1)).strftime("%Y-%m-%d")
# DATE_TO   = "2025-07-18"#datetime.now().strftime("%Y-%m-%d")

# # ────────────────────────────────────────────────────────────
# # 2) SESSION
# # ────────────────────────────────────────────────────────────
# session = requests.Session()
# session.auth = (API_KEY, API_SECRET)
# session.headers.update({"Authorization": f"{API_KEY}:{API_SECRET}"})
# REQUEST_TIMEOUT = 15     # seconds

# # ────────────────────────────────────────────────────────────
# # 3) Rate-limit helpers
# # ────────────────────────────────────────────────────────────
# def secs_from_header(raw: str | None) -> int | None:
#     """Return number of seconds to wait, or None if header absent/invalid."""
#     if not (raw and raw.isdigit()):
#         return None
#     val = int(raw)
#     return val - int(time.time()) if val > 86_400 else val   # epoch vs delta

# def respect_burst(window_start, calls_made):
#     """Block if we already made MAX_CALLS_PER_MIN requests in this minute."""
#     if calls_made and calls_made % MAX_CALLS_PER_MIN == 0:
#         elapsed = time.time() - window_start
#         wait = max(0, 60 - elapsed)
#         if wait:
#             print(f"↪︎ pacing – sleeping {wait:.1f}s to stay under burst limit")
#             time.sleep(wait)
#         return time.time()   # reset window start
#     return window_start

# # ────────────────────────────────────────────────────────────
# # 4) GET with smart 429 handling
# # ────────────────────────────────────────────────────────────
# def safe_get(url):
#     for attempt in range(MAX_RETRIES):
#         r = session.get(url, timeout=REQUEST_TIMEOUT)
#         if r.status_code != 429:
#             r.raise_for_status()
#             return r.json()

#         wait = secs_from_header(r.headers.get("X-Rate-Limit-Burst-Reset") or
#                                 r.headers.get("Retry-After"))
#         if wait is None:
#             wait = BACKOFF_FACTOR ** attempt
#         wait = max(wait, 1)
#         print(f"429 → wait {wait}s (retry {attempt+1}/{MAX_RETRIES})")
#         time.sleep(wait)
#     raise RuntimeError(f"gave up after {MAX_RETRIES} retries → {url}")

# # ────────────────────────────────────────────────────────────
# # 5) API wrappers
# # ────────────────────────────────────────────────────────────
# def list_calls(date_from=DATE_FROM, date_to=DATE_TO):
#     url = "https://api.justcall.io/v1/autodialer/calls/list"
#     all_calls, page = [], 1
#     while True:
#         payload = {"start_date": date_from, "end_date": date_to, "page": page}
#         r = session.post(url, json=payload, timeout=REQUEST_TIMEOUT)
#         r.raise_for_status()
#         data = r.json().get("data", [])
#         if not data:
#             break
#         all_calls.extend(data)
#         print(f"• got page {page} ({len(data)} calls)")
#         page += 1
#     return all_calls

# def call_detail(call_id):
#     url = f"https://api.justcall.io/v2.1/sales_dialer/calls/{call_id}"
#     return safe_get(url).get("data", {})

# # ────────────────────────────────────────────────────────────
# # 6) Flatten to DataFrame   ← drop‑in replacement
# # ────────────────────────────────────────────────────────────
# def flatten(detail_list):
#     """Return a DataFrame with ALL documented Sales‑Dialer fields."""
#     today = datetime.utcnow().strftime("%Y-%m-%d")

#     rows = []
#     for d in detail_list:
#         # Level‑1
#         row = {
#             "call_id"             : d.get("call_id"),
#             "call_sid"            : d.get("call_sid"),
#             "contact_id"          : d.get("contact_id"),
#             "contact_number"      : d.get("contact_number"),
#             "contact_name"        : d.get("contact_name"),
#             "contact_email"       : d.get("contact_email"),
#             "sales_dialer_number" : d.get("sales_dialer_number"),
#             "agent_id"            : d.get("agent_id"),
#             "agent_name"          : d.get("agent_name"),
#             "agent_email"         : d.get("agent_email"),
#             "call_date"           : d.get("call_date"),
#             "call_user_date"      : d.get("call_user_date"),
#             "call_time"           : d.get("call_time"),
#             "call_user_time"      : d.get("call_user_time"),
#             "date_ingested"       : today,
#         }

#         # ── campaign object ──────────────────────────────────
#         camp = d.get("campaign", {}) or {}
#         row.update({
#             "campaign_id"  : camp.get("id"),
#             "campaign_name": camp.get("name"),
#             "campaign_type": camp.get("type"),
#             # keep full JSON in case spec expands
#             "campaign_raw" : json.dumps(camp, default=str),
#         })

#         # ── call_info object ────────────────────────────────
#         info = d.get("call_info", {}) or {}
#         row.update({
#             "reattempt_number"     : info.get("reattempt_number"),
#             "cost_incurred"        : info.get("cost_incurred"),
#             "call_answered_by"     : info.get("call_answered_by"),
#             "direction"            : info.get("direction"),
#             "direction_type"       : info.get("type"),
#             "duration"             : info.get("duration"),
#             "friendly_duration"    : info.get("friendly_duration"),
#             "disposition"          : info.get("disposition"),
#             "notes"                : info.get("notes"),
#             "rating"               : info.get("rating"),
#             "recording_url"        : info.get("recording"),
#             # JSON stash
#             "call_info_raw"        : json.dumps(info, default=str),
#         })

#         # ── justcall_ai object ───────────────────────────────
#         ai = d.get("justcall_ai", {}) or {}
#         row.update({
#             "call_moments"           : json.dumps(ai.get("call_moments", [])),
#             "customer_sentiment"     : ai.get("customer_sentiment"),
#             "call_score"             : ai.get("call_score"),
#             "manual_call_score"      : ai.get("manual_call_score"),
#             "call_summary"           : ai.get("call_summary"),
#         })

#         # nested score parameters
#         scores = (ai.get("call_score_parameters") or {})
#         row.update({
#             "score_dead_air_time"         : scores.get("dead_air_time"),
#             "score_filler_word"           : scores.get("filler_word"),
#             "score_de_escalation"         : scores.get("de_escalation"),
#             "score_empathy"               : scores.get("empathy"),
#             "score_talk_listen_ratio"     : scores.get("talk_listen_ratio"),
#             "score_greetings"             : scores.get("greetings"),
#             "score_words_per_minute"      : scores.get("words_per_minute"),
#             "score_monologue_duration"    : scores.get("monologue_duration"),
#             "score_call_etiquette"        : scores.get("call_etiquette"),
#             "score_customer_sentiment"    : scores.get("customer_sentiment_score"),
#             # keep raw in case fields change
#             "score_params_raw"            : json.dumps(scores, default=str),
#         })

#         # tags (array) – store as JSON string so PG → jsonb later if desired
#         row["tags"] = json.dumps(ai.get("tags", []))

#         rows.append(row)

#     # create tidy DataFrame with stable column order
#     df = pd.DataFrame(rows)
#     return df.reindex(sorted(df.columns), axis=1)


# # ────────────────────────────────────────────────────────────
# # 7) Main workflow
# # ────────────────────────────────────────────────────────────
# def run_ingestion():
#     print(f"\n⏳ Fetching IDs {DATE_FROM} → {DATE_TO} …")
#     ids = list_calls()
#     total = len(ids)
#     print(f"✓ {total} IDs found\n")

#     details, missing = [], []
#     window_start = time.time()

#     for idx, c in enumerate(ids, 1):
#         window_start = respect_burst(window_start, idx)
#         cid = c["call_id"]
#         try:
#             d = call_detail(cid)
#             if d:
#                 details.append(d)
#             else:
#                 missing.append(cid)
#             if idx % 25 == 0 or idx == total:
#                 print(f"  progress {idx}/{total}")
#         except Exception as e:
#             print(f"⚠️  {cid} skipped → {e}")
#             missing.append(cid)

#     df = flatten(details)
#     print(f"\n🏁 finished – {len(df)}/{total} rows")
#     if missing:
#         print(f"  still missing {len(missing)} IDs → {missing[:10]} …")
#     return df

# # ────────────────────────────────────────────────────────────
# if __name__ == "__main__":
#     df_calls = run_ingestion()
#     # df_calls.to_csv("justcall_calls.csv", index=False)



⏳ Fetching IDs 2025-07-18 → 2025-07-18 …
• got page 1 (50 calls)
• got page 2 (50 calls)
• got page 3 (50 calls)
• got page 4 (50 calls)
• got page 5 (50 calls)
• got page 6 (50 calls)
• got page 7 (50 calls)
• got page 8 (50 calls)
• got page 9 (50 calls)
• got page 10 (50 calls)
• got page 11 (50 calls)
• got page 12 (50 calls)
• got page 13 (50 calls)
• got page 14 (50 calls)
• got page 15 (50 calls)
• got page 16 (30 calls)
✓ 780 IDs found

  progress 25/780
↪︎ pacing – sleeping 57.5s to stay under burst limit
  progress 50/780
↪︎ pacing – sleeping 57.6s to stay under burst limit
  progress 75/780
↪︎ pacing – sleeping 57.5s to stay under burst limit
  progress 100/780
↪︎ pacing – sleeping 57.2s to stay under burst limit
  progress 125/780
↪︎ pacing – sleeping 57.6s to stay under burst limit
  progress 150/780
↪︎ pacing – sleeping 57.5s to stay under burst limit
  progress 175/780
↪︎ pacing – sleeping 57.5s to stay under burst limit
  progress 200/780
↪︎ pacing – sleeping 57.7s to 

In [3]:
display(df_calls)

,agent_email,agent_id,agent_name,call_answered_by,call_date,call_id,call_info_raw,call_moments,call_score,call_sid,...,score_de_escalation,score_dead_air_time,score_empathy,score_filler_word,score_greetings,score_monologue_duration,score_params_raw,score_talk_listen_ratio,score_words_per_minute,tags
0,lorie.jane@gushwork.ai,420268,Lorie Jane Salpid,Human,2025-07-17,87909065,"{""reattempt_number"": 0, ""cost_incurred"": 0.024...",[],None,CA4dfdc89c075cf94df56f8bfc263f940a,...,None,None,None,None,None,None,{},None,None,[]
1,ken.david@gushwork.ai,421713,Ken David,Human,2025-07-17,87909156,"{""reattempt_number"": 0, ""cost_incurred"": 0.026...",[],None,CA05b63ce6cc5c83701ee4a6672fcd455b,...,None,None,None,None,None,None,{},None,None,[]
2,ken.david@gushwork.ai,421713,Ken David,Human,2025-07-17,87909287,"{""reattempt_number"": 0, ""cost_incurred"": 0.024...",[],None,CA2d2673c65f835d8db18524607d269aa1,...,None,None,None,None,None,None,{},None,None,[]
3,lorie.jane@gushwork.ai,420268,Lorie Jane Salpid,Human,2025-07-17,87909372,"{""reattempt_number"": 0, ""cost_incurred"": 0.024...",[],None,CAfadf53817b829e7715a3ac48a2ff01d9,...,None,None,None,None,None,None,{},None,None,[]
4,lorie.jane@gushwork.ai,420268,Lorie Jane Salpid,Human,2025-07-17,87909833,"{""reattempt_number"": 0, ""cost_incurred"": 0.024...",[],None,CA15fb7ae215078b2b206d1efa0d703dcb,...,None,None,None,None,None,None,{},None,None,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
775,lorie.jane@gushwork.ai,420268,Lorie Jane Salpid,Human,2025-07-18,88028420,"{""reattempt_number"": 0, ""cost_incurred"": 0.024...",[],None,CA515baf19fc03ff659f567609fd81346d,...,None,None,None,None,None,None,{},None,None,[]
776,ken.david@gushwork.ai,421713,Ken David,Human,2025-07-18,88028433,"{""reattempt_number"": 0, ""cost_incurred"": 0.024...",[],None,CA0300dda2d66fa99683431f7adb9f6cef,...,None,None,None,None,None,None,{},None,None,[]
777,ken.david@gushwork.ai,421713,Ken David,Human,2025-07-18,88028542,"{""reattempt_number"": 0, ""cost_incurred"": 0.024...",[],None,CA5294263b02d189fcbf2287ce6e4b1c5e,...,None,None,None,None,None,None,{},None,None,[]
778,lorie.jane@gushwork.ai,420268,Lorie Jane Salpid,Human,2025-07-18,88028621,"{""reattempt_number"": 0, ""cost_incurred"": 0.048...",[],None,CAc6a01c327659188a8b171873f061e035,...,None,None,None,None,None,None,{},None,None,[]
